In [1]:
import pandas as pd
import requests
import json
from sodapy import Socrata

## Background

## Process

In [2]:
# First, get your Census API key from here: https://api.census.gov/data/key_signup.html

apiKey = "b4938dd4ca4123f1125e3afca615defa6b258b80"

# Then, get your Socrata API key from here (or download and import the datasets):

## 1. Use the Socrata API to import dataset with PUMA info from NYC Open Data Portal

In [3]:
# import the census tract to PUMA dataset from the NYC Open Data Portal
client = Socrata("data.cityofnewyork.us", 'n2OlwDYEzVwYSJ5jnp2qJcNxW')
nta_nyc_od_link = client.get("8ius-dhrr", limit=5000)

nyc_geo_equivalency = pd.DataFrame(nta_nyc_od_link)
# adding leading zeroes to make it compatible with other datasets
nyc_geo_equivalency["_2010_census_tract"] = nyc_geo_equivalency['_2010_census_tract'].map(lambda x: '{0:0>6}'.format(x))
nyc_geo_equivalency["_2010_census_bureau_fips_county_code"] = nyc_geo_equivalency['_2010_census_bureau_fips_county_code'].map(lambda x: '{0:0>3}'.format(x))

In [4]:
nyc_geo_equivalency['_2010_census_bureau_fips_county_code'].value_counts()

047    761
081    669
005    339
061    288
085    111
Name: _2010_census_bureau_fips_county_code, dtype: int64

In [5]:
# Reorganize the data at the PUMA level

puma_attributes = nyc_geo_equivalency.groupby(['borough', 'puma'])['neighborhood_tabulation_area_nta_name'].unique().reset_index()

# clean NTA name
# nta_name_str_list = []
# for i in puma_attributes['neighborhood_tabulation_area_nta_name']:
#     nta_name_string = ', '.join(i)
#     nta_name_str_list.append(nta_name_string)
# puma_attributes['nta_names_combined_str'] =nta_name_str_list
puma_attributes = puma_attributes.drop(columns=['neighborhood_tabulation_area_nta_name'])

In [6]:
# add a new column called 'PUMA FIPS Code' in the puma attributes df
puma_attributes['PUMA FIPS'] = '7950000US360' + puma_attributes['puma'].astype(str)

In [7]:
#there are 55 PUMA's in the city
puma_attributes.shape

(55, 3)

In [8]:
puma_attributes.head()

,borough,puma,PUMA FIPS
0,Bronx,3701,7950000US3603701
1,Bronx,3702,7950000US3603702
2,Bronx,3703,7950000US3603703
3,Bronx,3704,7950000US3603704
4,Bronx,3705,7950000US3603705


## 2. Identify LEP and CVALEP Residents in NYC

### 2.1 Identify LEP residents in each PUMA (roughly approximated to a Community District)

#### There are three variables to identify LEPs:
- Language Other than English Spoken at Home (LANP)
- Age (AGE) = +5
- Ability to Speak English (ENG). Any level except "Very Well" qualifies as a limited english proficient.
    - 2 = Well
    - 3 = Not Well
    - 4 = Not At All

PWGTP is the Person Weight to take into account the number of persons in a household.

In [9]:
#the limit on the PUMS data go to 100
age_string = ''.join([('&AGEP=' + str(i)) for i in range(10, 100)])

# concat puma codes into a single string
puma_str = ','.join(puma_attributes['PUMA FIPS'])

In [10]:
# using Census API, import the data for LEPs

# 5+ years old Residents who speak English less than "Very Well"
# Source ACS 5 Year 2015-2019 Public Use Microdata Sample
lep_API_summary = "https://api.census.gov/data/2019/acs/acs5/pums?tabulate=weight(PWGTP)&col+LANP&row+ucgid&ucgid=" + puma_str + "&AGEP=05&AGEP=06&AGEP=07&AGEP=08&AGEP=09" + age_string + "&ENG=2&ENG=3&ENG=4&key=" + apiKey

#call the API and collect the response
response_lep_summary = requests.get(lep_API_summary)

#load the response into a JSON
formattedResponse_lep_summary = json.loads(response_lep_summary.text)
lep_summary_df = pd.DataFrame(formattedResponse_lep_summary)

In [11]:
lep_summary_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,{'LANP': '9999'},{'LANP': '7300'},{'LANP': '7124'},{'LANP': '7060'},{'LANP': '7050'},{'LANP': '7039'},{'LANP': '7032'},{'LANP': '7019'},{'LANP': '6933'},{'LANP': '6930'},...,{'LANP': '999'},{'LANP': '7059'},{'LANP': '6936'},{'LANP': '1515'},{'LANP': '1603'},{'LANP': '1652'},{'LANP': '4750'},{'LANP': '5365'},{'LANP': '6450'},ucgid


Now using an input of language codes, rename the header row to reflect languages.


Before running this function, download the PUMS 5-year data dictionary.

For 2015-2019, the data dictionary is located here: https://www.census.gov/programs-surveys/acs/microdata/documentation/2019.html

In [12]:
def rename_header_languages(df, link_to_pums_data_dictionary):
    '''
    Rename columns to reflect languages. Download the 
    
    Input:
        df: a dataframe with data extracted from ACS PUMS 5-year.
        link_to_pums_data_dictionary: 
    
    Returns:
        df: input dataframe with column names that reflect languages.
        languages_code_dict: a dictionary where each language has a code.
    '''
    
    puma_data_dict_df = pd.read_csv(link_to_pums_data_dictionary)
    
    # find out the codes of the languages
    
    languages_codes_dict = puma_data_dict_df[puma_data_dict_df['RT'] == 'LANP'][['Record Type', 'Unnamed: 6']].iloc[2:, :].set_index('Record Type').to_dict()['Unnamed: 6']

    # change header row
    new_header = []
    #grab the first row for the header
    for k in df.iloc[0]:
        
        if type(k) == dict:
            
            lang_code = k['LANP']
            language = languages_codes_dict.get(lang_code)
            if language is None:
                # this is how Census assigns blank values to languages
                language = 'N/A (GQ/Vacant)'
            #print(language)
            new_header.append(language)
        else:
            #print(k)
            new_header.append(k)

    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header

    return df, languages_codes_dict

In [13]:
lep_summary_df_renamed, languages_code_dict = rename_header_languages(lep_summary_df, "//chnetappfs01/MODA/Project CEC Poll Site Selection/0. Understanding PUMS/PUMS_Data_Dictionary_2015-2019.csv")

In [14]:
# sort column names alphabetically

lep_summary_df_sorted = lep_summary_df_renamed.reindex(sorted(lep_summary_df_renamed.columns), axis=1)
_
lep_summary_df_sorted = lep_summary_df_sorted.rename(columns={'ucgid' : 'PUMA Code'})
lep_summary_df_sorted.head(1)

,Afrikaans,Akan (incl. Twi),Albanian,Aleut languages,Amharic,Apache languages,Arabic,Armenian,Assyrian Neo-Aramaic,Bengali,...,Ukrainian,Urdu,Uto-Aztecan languages,Uzbek,Vietnamese,Wolof,Yiddish,Yoruba,Zuni (2016 or earlier),PUMA Code
1,0,281,178,0,65,0,79,0,12,121,...,255,26,0,0,69,0,10,31,0,7950000US3603701


In [15]:
# combine Chinese, Mandarin and Cantonese together (note that when people fill out ACS forms, there's option to choose only one language)
lep_summary_df_sorted['Chinese (incl. Mandarin, Cantonese)'] = lep_summary_df_sorted['Chinese'] + lep_summary_df_sorted['Mandarin'] + lep_summary_df_sorted['Cantonese']

#drop the individual level Chinese, Mandarin and Cantonese languages
lep_summary_df_sorted = lep_summary_df_sorted.drop(columns=['Chinese', 'Mandarin', 'Cantonese'])


Now join the NYC LEP Dataframe at the PUMA level with the PUMA data to get geographical attributes

In [16]:
lep_summary_puma_attrs = pd.merge(lep_summary_df_sorted, puma_attributes, left_on = 'PUMA Code', right_on = 'PUMA FIPS', how='left').iloc[:, :-1]

In [17]:
lep_summary_puma_attrs.head(1)

,Afrikaans,Akan (incl. Twi),Albanian,Aleut languages,Amharic,Apache languages,Arabic,Armenian,Assyrian Neo-Aramaic,Bengali,...,Uzbek,Vietnamese,Wolof,Yiddish,Yoruba,Zuni (2016 or earlier),PUMA Code,"Chinese (incl. Mandarin, Cantonese)",borough,puma
0,0,281,178,0,65,0,79,0,12,121,...,0,69,0,10,31,0,7950000US3603701,444,Bronx,3701


Melt the dataframe so that at each row is a language in a specific PUMA (ordered by language)

In [18]:
lep_summary_puma_attrs_melt = lep_summary_puma_attrs.melt(id_vars=['borough', 'puma', 'PUMA Code']).rename(columns={'value' : 'LEP', 'variable' : 'Language'})

In [19]:
lep_summary_puma_attrs_melt.head(2)

,borough,puma,PUMA Code,Language,LEP
0,Bronx,3701,7950000US3603701,Afrikaans,0
1,Bronx,3702,7950000US3603702,Afrikaans,0


### 2.2 Identify CVALEP residents in each PUMA (roughly equivalent to a Community District)

#### There are four variables to identify CVALEPs:
- Language Other than English Spoken at Home (LANP)
- Age (AGE) = +18 (as opposed to +5 when it came to identifying the LEPs)
- Ability to Speak English (ENG). Any level except "Very Well" qualifies as a limited english proficient.
    - 2 = Well
    - 3 = Not Well
    - 4 = Not At All
- Citizen Status (CIT)
    - 1 = Born in the US
    - 2 = Born in Puerto Rico, Guam, the US Virgin Islands
    - 3 = Board abroad of American parents
    - 4 = U.S. Citizen by Naturalization

In [20]:
age_string_18_plus = ''.join([('&AGEP=' + str(i)) for i in range(18, 100)])

In [21]:
# 18+ years old Residents who speak English less than "Very Well" and are US Citizens (CVALEP)

# Source ACS 5 Year 2015-2019 Public Use Microdata Sample
cvalep_API_summary = "https://api.census.gov/data/2019/acs/acs5/pums?tabulate=weight(PWGTP)&col+LANP&row+ucgid&ucgid=" + puma_str + age_string_18_plus + "&CIT=1&CIT=2&CIT=3&CIT=4&ENG=2&ENG=3&ENG=4&key=" + apiKey

#call the API and collect the response
response_cvalep_summary = requests.get(cvalep_API_summary)

#load the response into a JSON
formattedResponse_cvalep_summary = json.loads(response_cvalep_summary.text)

In [22]:
cvalep_summary_df = pd.DataFrame(formattedResponse_cvalep_summary)

Now using an input of language codes, rename the header row to reflect languages.


Before running this function, download the PUMS 5-year data dictionary from here (you may have already downloaded one above)

For 2015-2019, the data dictionary is located here: https://www.census.gov/programs-surveys/acs/microdata/documentation/2019.html

In [23]:
cvalep_summary_df_renamed, languages_code_dict = rename_header_languages(cvalep_summary_df, "//chnetappfs01/MODA/Project CEC Poll Site Selection/0. Understanding PUMS/PUMS_Data_Dictionary_2015-2019.csv")

In [24]:
# change column names alphabetically
cvalep_summary_df_sorted = cvalep_summary_df_renamed.reindex(sorted(cvalep_summary_df_renamed.columns), axis=1)

# combine chinese, mandarin and cantonese together
cvalep_summary_df_sorted['Chinese (incl. Mandarin, Cantonese)'] = cvalep_summary_df_sorted['Chinese'] + cvalep_summary_df_sorted['Mandarin'] + cvalep_summary_df_sorted['Cantonese']

# drop Chinese, Mandarin and Cantonese
cvalep_summary_df_sorted = cvalep_summary_df_sorted.drop(columns=['Chinese', 'Mandarin', 'Cantonese'])

cvalep_summary_df_sorted = cvalep_summary_df_sorted.rename(columns={'ucgid' : 'PUMA Code'})

Merge NYC CVALEP data with the PUMA data to get geographical attributes

In [25]:

cvalep_summary_puma_attrs = pd.merge(cvalep_summary_df_sorted, puma_attributes, left_on = 'PUMA Code', right_on = 'PUMA FIPS', how='left').iloc[:, :-1]

In [26]:
cvalep_summary_puma_attrs_melt = cvalep_summary_puma_attrs.melt(id_vars=['borough', 'puma', 'PUMA Code']).rename(columns={'value' : 'CVALEP', 'variable' : 'Language'})

Now, merge the LEP data and the CVALEP data together

In [27]:
# this is before the BOE
lep_and_cvalep_merged = lep_summary_puma_attrs_melt.merge(cvalep_summary_puma_attrs_melt, on =['borough', 'PUMA Code','puma', 'Language'], how='inner', indicator=True)

In [28]:
lep_and_cvalep_merged['_merge'].value_counts()

both          7535
right_only       0
left_only        0
Name: _merge, dtype: int64

In [29]:
#what is the proportion of CVALEPs out of LEP residents? Over 50%
lep_and_cvalep_merged['CVALEP'].sum() / lep_and_cvalep_merged['LEP'].sum()*100

50.33121522071453

## Step 3. Aggregate Data
- #### 1) CVALEP % at CD level and 
- #### 2) CVALEP % by Languages at Community District Level

Import the mapping dataset from the NYC Open Data Portal on which Community District belongs to which PUMA.

First, import this dataset from: [provide link here]

In [30]:
nyc_puma_to_cd = pd.read_excel("//chnetappfs01/MODA/Project CEC Poll Site Selection/5. Data/nyc2010census_tabulation_equiv (1).xlsx", sheet_name= "NTA in PUMA_", header=6).reset_index()

In [31]:
# clean the 
nyc_puma_to_cd = nyc_puma_to_cd.rename(columns={'level_0' : 'Borough', 
                                                'level_1' : '2010 Census Bureau FIPS County Code', \
                                                'level_2' : '2010 NYC Borough Code', \
                                                'PUMA \nCode' : 'PUMA Code', \
                                                'Community District \n(PUMAs approximate NYC Community  Districts and are not coterminous)' : 'Community District', \
                                                'NTA \nCode' : 'NTA Code',
                                               'Name' : 'NTA Name'})
                                       
nyc_puma_to_cd = nyc_puma_to_cd.drop(columns=['NTA Code', 'NTA Name']).groupby(['Borough',  '2010 NYC Borough Code', 'PUMA Code', 'Community District']).size().reset_index().drop(columns=0)
nyc_puma_to_cd['PUMA Code'] = nyc_puma_to_cd['PUMA Code'].astype(int)

In [32]:
nyc_puma_to_cd.head()

,Borough,2010 NYC Borough Code,PUMA Code,Community District
0,Bronx,2,3701,BX Community District 8
1,Bronx,2,3702,BX Community District 12
2,Bronx,2,3703,BX Community District 10
3,Bronx,2,3704,BX Community District 11
4,Bronx,2,3705,BX Community Districts 3 & 6


Not all PUMA's can be mapped 1-to-1 with the CD's. There are 59 CD's and 55 PUMA's.

**Question: how do you distribute 4 PUMA's to the 8 CD's?**

One way to do that is by using population proportion i.e. allocate PUMA data to multiple CD's based on how much their total population falls with the PUMA.

Download the socrata API key from here or directly download and import the dataset into this Notebook: 

In [33]:
def get_nyc_population_cd_level(cd_nyc_od_url, api_key):
    '''
    Using Socrata API, import the dataset containing URL of the dataset that contains info on Census Decennial
    population at the Community District level.
    
    Input:
        cd_nyc_od_url: a string containing the 4x4 link of the open data portal
    
    Returns:
        A dataframe containing the data on population for each of the 59 community district.
    '''
    
    # first import the dataset on CD population
    client = Socrata("data.cityofnewyork.us", api_key)
    cd_nyc_od_link = client.get(cd_nyc_od_url, limit=5000)

    cd_nyc_population = pd.DataFrame(cd_nyc_od_link)

    # only 2010
    cd_nyc_population_only_2010 = cd_nyc_population.drop(columns=['_1970_population', '_1980_population', '_1990_population', '_2000_population'])
    cd_nyc_population_only_2010 = cd_nyc_population_only_2010.rename(columns={'_2010_population' : 'CD 2010 Population'})
    cd_nyc_population_only_2010['CD 2010 Population'] = cd_nyc_population_only_2010['CD 2010 Population'].astype(int)
    
    return cd_nyc_population_only_2010

In [34]:
# get nyc population at the CD level
cd_nyc_population_only_2010 = get_nyc_population_cd_level("xi7c-iiu2", 'n2OlwDYEzVwYSJ5jnp2qJcNxW')

In [35]:
def separate_out_cd_lines(df_with_cd_info):
    '''
    Create an equivalent number of duplicate rows when a PUMA has two CD's. These duplicate rows will be used to identify 
    data at the CD level for those two CD's in that PUMA (using population apportionment).
    
    Input:
        df_with_cd_info: A dataframe containing the LEP and CVALEP data at the PUMA level along with CD attribute for each PUMA.
    
    Output:
        A dataframe that duplicates the rows of the PUMA that have two CD's.
    
    '''
    
    all_rows_list = []
    puma_codes_list = []

    for idx, row in df_with_cd_info.iterrows():

        # if a PUMA has multiple CD's
        if "&" in row['Community District']:
            # get codes list
            cd_codes_list = [int(row['Community District'][-5]), int(row['Community District'][-1])] 
            
            puma_codes_list.append(row['puma'])
            cd_desc = row['Community District']

            # create extra rows so that each community district has the same set of rows/data as its counterpart in the same PUMA
            for code_ in cd_codes_list:
                row['Community District'] = cd_desc[:-7] + ' ' + str(code_)
                all_rows_list.append(list(row))


    new_columns = df_with_cd_info.columns.tolist()
    puma_by_cd_complete = pd.DataFrame(all_rows_list, columns=new_columns)
    puma_by_cd_complete['Borough'] = puma_by_cd_complete['Borough'].astype(str)


    # remove the rows with existing puma's from the df: 
    lep_and_cvalep_cd_info_clean = df_with_cd_info[~df_with_cd_info['puma'].isin(puma_codes_list)]
    lep_and_cvalep_cd_info_clean_full = pd.concat([lep_and_cvalep_cd_info_clean, puma_by_cd_complete], axis=0)
    
    return lep_and_cvalep_cd_info_clean_full

Merge the LEP and CVALEP data with the PUMA to CD geogrphical attributes

In [36]:
lep_and_cvalep_merged['puma'] = lep_and_cvalep_merged['puma'].astype(int)

In [37]:
lep_and_cvalep_puma_and_cd_info = pd.merge(lep_and_cvalep_merged, nyc_puma_to_cd, left_on='puma', right_on='PUMA Code', how='left')

In [38]:
lep_and_cvalep_puma_and_cd_info.head()

,borough,puma,PUMA Code_x,Language,LEP,CVALEP,_merge,Borough,2010 NYC Borough Code,PUMA Code_y,Community District
0,Bronx,3701,7950000US3603701,Afrikaans,0,0,both,Bronx,2,3701,BX Community District 8
1,Bronx,3702,7950000US3603702,Afrikaans,0,0,both,Bronx,2,3702,BX Community District 12
2,Bronx,3703,7950000US3603703,Afrikaans,0,0,both,Bronx,2,3703,BX Community District 10
3,Bronx,3704,7950000US3603704,Afrikaans,0,0,both,Bronx,2,3704,BX Community District 11
4,Bronx,3705,7950000US3603705,Afrikaans,0,0,both,Bronx,2,3705,BX Community Districts 3 & 6


In [44]:
lep_and_cvalep_puma_and_cd_info.shape

(7535, 11)

For rows (PUMA) that have two community districts (e.g. 3705 in Bronx), create a duplicate row for that PUMA for each language.

In [45]:
lep_and_cvalep_clean_full = separate_out_cd_lines(lep_and_cvalep_puma_and_cd_info)

In [46]:
lep_and_cvalep_clean_full.shape

(8083, 11)

In [47]:
def create_by_cd_analysis(lep_and_cvalep_clean_full, cd_nyc_population_only_2010):
    '''
    In the PUMA by Language dataframe, split the 4 PUMA's into 8 CD's and get LEP and CVALEP data at the CD level.
    
    Input:
        lep_and_cvalep_clean_full: A dataframe containing LEP and CVALEP data at the PUMA level with duplicate rows.
        cd_nyc_population_only_2010: A dataframe containing data from Open Data Portal on the Decennial Population of Community Districts.
        
    Output:
        Returns the dataframe containing the LEP and CVALEP data at the Community District level (instead of the PUMA level)
        
    '''    

    # for each CD, get their CD Code (so that we can join it with CD Decennial Population data on the Open Data Portal)
    lep_and_cvalep_clean_full['CD Code'] = lep_and_cvalep_clean_full['Community District'].str[-2:]
    lep_and_cvalep_clean_full['CD Code'] = lep_and_cvalep_clean_full['CD Code'].str.lstrip()
    lep_and_cvalep_clean_full = lep_and_cvalep_clean_full.reset_index(drop='index')
    
    # merge the LEP and CVALEP data with the CD Decennial population data to get population proportions for multiple CD's in a single PUMA 
    lep_and_cvalep_cd_pop = pd.merge(lep_and_cvalep_clean_full, cd_nyc_population_only_2010, left_on=['Borough', 'CD Code'], right_on = ['borough', 'cd_number'], how='left')
    
    #Get sum of puma population
    puma_temp1 = lep_and_cvalep_cd_pop[~lep_and_cvalep_cd_pop['CD 2010 Population'].duplicated()]
    puma_2010_population = puma_temp1.groupby(['puma'])['CD 2010 Population'].sum().reset_index(name='PUMA 2010 Population')
    
    # Merge LEP and CVALEP data with the PUMA's population
    lep_and_cvalep_cd_puma_pop = pd.merge(lep_and_cvalep_cd_pop, puma_2010_population, on='puma', how='left')
    
    # now do apportionment for the four CD's
    lep_and_cvalep_cd_puma_pop['% of PUMA 2010 Population'] = lep_and_cvalep_cd_puma_pop['CD 2010 Population'] / lep_and_cvalep_cd_puma_pop['PUMA 2010 Population']

    lep_and_cvalep_cd_puma_pop['Total LEP Population Estimate'] = lep_and_cvalep_cd_puma_pop['LEP'] * lep_and_cvalep_cd_puma_pop['% of PUMA 2010 Population']

    lep_and_cvalep_cd_puma_pop['Total CVALEP Population Estimate'] = lep_and_cvalep_cd_puma_pop['CVALEP'] * lep_and_cvalep_cd_puma_pop['% of PUMA 2010 Population']

    # add one more column i.e. Borough CD Code
    # first add a leading zero to cd_number
    lep_and_cvalep_cd_puma_pop['CD Code leading zero'] = lep_and_cvalep_cd_puma_pop['CD Code'].apply(lambda x: x.zfill(2))
    lep_and_cvalep_cd_puma_pop['Borough CD Code'] = lep_and_cvalep_cd_puma_pop['2010 NYC Borough Code'].astype(str) + lep_and_cvalep_cd_puma_pop['CD Code leading zero'].astype(str)

    lep_and_cvalep_cd_puma_pop = lep_and_cvalep_cd_puma_pop.rename(columns= {'cd_name' : 'Community District Name'})
    
    return lep_and_cvalep_cd_puma_pop

In [48]:
lep_and_cvalep_cd_puma_pop = create_by_cd_analysis(lep_and_cvalep_clean_full, cd_nyc_population_only_2010)

In [49]:
lep_and_cvalep_cd_puma_pop['Total CVALEP Population Estimate'].sum()

894888.0

In [50]:
lep_and_cvalep_cd_puma_pop['Total LEP Population Estimate'].sum()

1777998.0

In [51]:
lep_and_cvalep_cd_puma_pop.shape

(8083, 22)

## Step 4. Export Aggregated Data

#### Print % of LEP and CVALEP for each CD (Overall)

In [52]:
# first aggregate the data from language level to the community district level
lep_and_cvalep_by_cd_overall = lep_and_cvalep_cd_puma_pop.groupby(['Borough', 'CD Code', 'Borough CD Code', 'Community District', 'Community District Name']).agg({'Total CVALEP Population Estimate' : 'sum', 'Total LEP Population Estimate' : 'sum'}).reset_index()
lep_and_cvalep_by_cd_overall_with_population = pd.merge(lep_and_cvalep_by_cd_overall, cd_nyc_population_only_2010, left_on=['Borough', 'CD Code'], right_on=['borough', 'cd_number'])

# add new columns i.e. % of population that is LEP and CVALEP
lep_and_cvalep_by_cd_overall_with_population['% of Population that is LEP'] = (lep_and_cvalep_by_cd_overall_with_population['Total LEP Population Estimate'] / lep_and_cvalep_by_cd_overall_with_population['CD 2010 Population']) * 100
lep_and_cvalep_by_cd_overall_with_population['% of Population that is CVALEP'] = (lep_and_cvalep_by_cd_overall_with_population['Total CVALEP Population Estimate'] / lep_and_cvalep_by_cd_overall_with_population['CD 2010 Population']) * 100

# filter only to the columns that are required
lep_and_cvalep_cd_puma_pop_for_print_overall = lep_and_cvalep_by_cd_overall_with_population[['Borough', 'Borough CD Code', 'Community District', 'Community District Name', 'CD 2010 Population', 'Total LEP Population Estimate', '% of Population that is LEP', 'Total CVALEP Population Estimate', '% of Population that is CVALEP']]
lep_and_cvalep_cd_puma_pop_for_print_overall = lep_and_cvalep_cd_puma_pop_for_print_overall.sort_values('Borough CD Code').reset_index(drop=True)
print(lep_and_cvalep_cd_puma_pop_for_print_overall.dtypes)
# Round off
lep_and_cvalep_cd_puma_pop_for_print_overall['Total LEP Population Estimate'] = lep_and_cvalep_cd_puma_pop_for_print_overall['Total LEP Population Estimate'].round().astype(int)
lep_and_cvalep_cd_puma_pop_for_print_overall['% of Population that is LEP'] = lep_and_cvalep_cd_puma_pop_for_print_overall['% of Population that is LEP'].round(1)

lep_and_cvalep_cd_puma_pop_for_print_overall['Total CVALEP Population Estimate'] = lep_and_cvalep_cd_puma_pop_for_print_overall['Total CVALEP Population Estimate'].round().astype(int)
lep_and_cvalep_cd_puma_pop_for_print_overall['% of Population that is CVALEP'] = lep_and_cvalep_cd_puma_pop_for_print_overall['% of Population that is CVALEP'].round(1)

# lep_and_cvalep_cd_puma_pop_for_print.to_csv("Table 1 - % of LEP and CVALEP by CD.csv")

Borough                              object
Borough CD Code                      object
Community District                   object
Community District Name              object
CD 2010 Population                    int32
Total LEP Population Estimate       float64
% of Population that is LEP         float64
Total CVALEP Population Estimate    float64
% of Population that is CVALEP      float64
dtype: object


In [53]:
lep_and_cvalep_cd_puma_pop_for_print_overall.head()

,Borough,Borough CD Code,Community District,Community District Name,CD 2010 Population,Total LEP Population Estimate,% of Population that is LEP,Total CVALEP Population Estimate,% of Population that is CVALEP
0,Manhattan,101,MN Community District 1,"Battery Park City, Tribeca",60978,3151,5.2,1789,2.9
1,Manhattan,102,MN Community District 2,"Greenwich Village, Soho",90016,4652,5.2,2642,2.9
2,Manhattan,103,MN Community District 3,"Lower East Side, Chinatown",163277,41906,25.7,26802,16.4
3,Manhattan,104,MN Community District 4,"Chelsea, Clinton",103245,8774,8.5,5252,5.1
4,Manhattan,105,MN Community District 5,Midtown Business District,51673,4391,8.5,2628,5.1


In [54]:
lep_and_cvalep_cd_puma_pop_for_print_overall.shape


(59, 9)

In [55]:
lep_and_cvalep_cd_puma_pop_for_print_overall['Total LEP Population Estimate'].sum()

1777998

In [56]:
lep_and_cvalep_cd_puma_pop_for_print_overall['Total CVALEP Population Estimate'].sum()

894888

#### Print % of LEP and CVALEP for each Community District by Language

In [57]:
# get the total LEP and CVALEP population for each community district
total_lep_by_CD = lep_and_cvalep_cd_puma_pop.groupby(['Borough CD Code'])['Total LEP Population Estimate'].sum().reset_index(name='Total LEP Population by CD')
total_cvalep_by_CD = lep_and_cvalep_cd_puma_pop.groupby(['Borough CD Code'])['Total CVALEP Population Estimate'].sum().reset_index(name='Total CVALEP Population by CD')

lep_and_cvalep_by_language_final = pd.merge(lep_and_cvalep_cd_puma_pop, total_lep_by_CD, on='Borough CD Code', how='left')
lep_and_cvalep_by_language_final_final = pd.merge(lep_and_cvalep_by_language_final, total_cvalep_by_CD, on ='Borough CD Code', how='left')

# use the info collected above to find out % of LEP Population and CVALEP population for each language in that Community District
lep_and_cvalep_by_language_final_final['% of LEP Population'] = (lep_and_cvalep_by_language_final_final['Total LEP Population Estimate'] / lep_and_cvalep_by_language_final_final['Total LEP Population by CD']) * 100
lep_and_cvalep_by_language_final_final['% of CVALEP Population'] = (lep_and_cvalep_by_language_final_final['Total CVALEP Population Estimate'] / lep_and_cvalep_by_language_final_final['Total CVALEP Population by CD']) * 100

lep_and_cvalep_cd_puma_pop_for_print_by_lang = lep_and_cvalep_by_language_final_final[['Borough', 'Borough CD Code', 'Community District', 'Community District Name', 'Language','Total LEP Population Estimate', '% of LEP Population', 'Total CVALEP Population Estimate', '% of CVALEP Population']]
lep_and_cvalep_cd_puma_pop_for_print_by_lang = lep_and_cvalep_cd_puma_pop_for_print_by_lang.sort_values(['Borough CD Code', 'Language']).reset_index(drop=True)

# Round off
lep_and_cvalep_cd_puma_pop_for_print_by_lang['Total LEP Population Estimate'] = lep_and_cvalep_cd_puma_pop_for_print_by_lang['Total LEP Population Estimate'].astype(float).round().astype(int)
lep_and_cvalep_cd_puma_pop_for_print_by_lang['% of LEP Population'] = lep_and_cvalep_cd_puma_pop_for_print_by_lang['% of LEP Population'].astype(float).round(1)

lep_and_cvalep_cd_puma_pop_for_print_by_lang['Total CVALEP Population Estimate'] = lep_and_cvalep_cd_puma_pop_for_print_by_lang['Total CVALEP Population Estimate'].astype(float).round().astype(int)
lep_and_cvalep_cd_puma_pop_for_print_by_lang['% of CVALEP Population'] = lep_and_cvalep_cd_puma_pop_for_print_by_lang['% of CVALEP Population'].astype(float).round(1)

#lep_and_cvalep_cd_puma_pop_for_print.to_csv("Table 2 - % of LEP Languages by CD.csv")

In [58]:
lep_and_cvalep_cd_puma_pop_for_print_by_lang.shape

(8083, 9)

In [59]:
lep_and_cvalep_cd_puma_pop_for_print_by_lang.head()

,Borough,Borough CD Code,Community District,Community District Name,Language,Total LEP Population Estimate,% of LEP Population,Total CVALEP Population Estimate,% of CVALEP Population
0,Manhattan,101,MN Community District 1,"Battery Park City, Tribeca",Afrikaans,0,0.0,0,0.0
1,Manhattan,101,MN Community District 1,"Battery Park City, Tribeca",Akan (incl. Twi),0,0.0,0,0.0
2,Manhattan,101,MN Community District 1,"Battery Park City, Tribeca",Albanian,7,0.2,0,0.0
3,Manhattan,101,MN Community District 1,"Battery Park City, Tribeca",Aleut languages,0,0.0,0,0.0
4,Manhattan,101,MN Community District 1,"Battery Park City, Tribeca",Amharic,0,0.0,0,0.0


In [60]:
lep_and_cvalep_cd_puma_pop_for_print_by_lang['Total LEP Population Estimate'].sum()

1777998

In [61]:
lep_and_cvalep_cd_puma_pop_for_print_by_lang['Total CVALEP Population Estimate'].sum()

894888